# Agentive Chatbot

In [6]:
import os
import time
import datetime
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display
from dotenv import load_dotenv
from semantic_router import Route
from semantic_router.encoders import OpenAIEncoder
from semantic_router.layer import RouteLayer
from openai import OpenAI
import yaml
from enum import Enum

load_dotenv('.env')
api_key = os.getenv("OPENAI_API_KEY")
encoder = OpenAIEncoder()

#base emotional states
agent_dir = "base emotional states"
emo_routes = []
emo_prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)
        if 'utterances' in agent_data:
            route = Route(
                name=agent_data['route_name'],
                utterances=agent_data['utterances']
            )
            emo_routes.append(route)
        if 'prompt' in agent_data:
            emo_prompts[agent_data['agent_name']] = agent_data['prompt']

#angry states
agent_dir = "angry_emotions"
angry_routes = []
angry_prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)
        if 'utterances' in agent_data:
            route = Route(
                name=agent_data['route_name'],
                utterances=agent_data['utterances']
            )
            angry_routes.append(route)
        if 'prompt' in agent_data:
            angry_prompts[agent_data['agent_name']] = agent_data['prompt']

#angry states
agent_dir = "anxiety_emotions"
anxiety_routes = []
anxiety_prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)
        if 'utterances' in agent_data:
            route = Route(
                name=agent_data['route_name'],
                utterances=agent_data['utterances']
            )
            anxiety_routes.append(route)
        if 'prompt' in agent_data:
            anxiety_prompts[agent_data['agent_name']] = agent_data['prompt']

#knowledge agents
agent_dir = "knowledge"
knowledge_routes = []
knowledge_prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)
        if 'utterances' in agent_data:
            route = Route(
                name=agent_data['route_name'],
                utterances=agent_data['utterances']
            )
            knowledge_routes.append(route)
        if 'prompt' in agent_data:
            knowledge_prompts[agent_data['agent_name']] = agent_data['prompt']

#memory agents
agent_dir = "memory"
memory_routes = []
memory_prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)
        if 'utterances' in agent_data:
            route = Route(
                name=agent_data['route_name'],
                utterances=agent_data['utterances']
            )
            memory_routes.append(route)
        if 'prompt' in agent_data:
            memory_prompts[agent_data['agent_name']] = agent_data['prompt']

#attitude and belief agents
agent_dir = "attitudes"
attitude_routes = []
attitude_prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)
        if 'utterances' in agent_data:
            route = Route(
                name=agent_data['route_name'],
                utterances=agent_data['utterances']
            )
            attitude_routes.append(route)
        if 'prompt' in agent_data:
            attitude_prompts[agent_data['agent_name']] = agent_data['prompt']


#routers
base_emotion_rl = RouteLayer(encoder = encoder, routes = emo_routes)
angry_emotion_rl = RouteLayer(encoder = encoder, routes = angry_routes)
anxiety_emotion_rl = RouteLayer(encoder = encoder, routes = anxiety_routes)
knowledge_rl = RouteLayer(encoder = encoder, routes = knowledge_routes)
memory_rl = RouteLayer(encoder = encoder, routes = memory_routes)
attitude_rl = RouteLayer(encoder = encoder, routes = attitude_routes)

#### Variables to store the default system prompt and scenario prompt

In [7]:
with open('syst_prompt.txt', 'r') as f:
    system_prompt = f.read()

with open('scenario_prompt.txt', 'r') as f:
    scenario_prompt = f.read()

#### Defines logic for updating, activating, and deactivating prompts

In [ ]:
CURRENT_BASE_STATE = 'ok'

def establish_base_state(prompt: str) -> None:
    """ 
    Establishes the basic emotion 
    Should only be called when current base state is 'ok' 
    """
    global CURRENT_BASE_STATE
    r = base_emotion_rl(prompt)
    CURRENT_BASE_STATE = r.name

def route_to_second_emotion(prompt: str) -> None:
    """ 
    Calls a function to update emotion based on the current emotion
    """
    if CURRENT_BASE_STATE == 'anxiety':
        prompt, question_category = update_anxiety_prompt(prompt)
    elif CURRENT_BASE_STATE == 'angry':
        prompt, question_category = update_angry_prompt(prompt)
    return prompt, question_category

CURRENT_SUB_STATE = 'ok'

class ANGRYSTATES(Enum):
    OK = 0
    ANNOYED = 1
    FRUSTRATED = 2
    ANGRY = 3
    ENRAGED = 4

def state_to_prompt(state: ANGRYSTATES) -> str:
    state_mapping = {
        ANGRYSTATES.OK: "ok",
        ANGRYSTATES.ANNOYED: "annoyed",
        ANGRYSTATES.FRUSTRATED: "frustrated",
        ANGRYSTATES.ANGRY: "angry",
        ANGRYSTATES.ENRAGED: "enraged",
    }
    return state_mapping[state]

def classify_angry_emotion(prompt: str) -> str:
    r = angry_emotion_rl(prompt)
    return r.name

def update_angry_prompt(user_input: str) -> str:
    global CURRENT_SUB_STATE
    global CURRENT_BASE_STATE
    question_category = classify_angry_emotion(user_input)
    print(f"emotional sub route: {question_category}")
    prompt = ""
    if question_category == 'escalate':
        if CURRENT_SUB_STATE != ANGRYSTATES.ENRAGED:
            CURRENT_SUB_STATE = ANGRYSTATES(CURRENT_SUB_STATE.value + 1)
    elif question_category == 'de_escalate':
        if CURRENT_SUB_STATE != ANGRYSTATES.OK:
            CURRENT_SUB_STATE = ANGRYSTATES(CURRENT_SUB_STATE.value - 1)
    prompt_key = state_to_prompt(CURRENT_SUB_STATE)
    prompt = emo_prompts[prompt_key]
    if CURRENT_SUB_STATE == ANGRYSTATES.OK:
        CURRENT_BASE_STATE = 'ok'
    return prompt, question_category

class ANXIETYSTATES(Enum):
    OK = 0
    UNEASY = 1
    CONCERNED = 2
    WORRIED = 3
    ANXIOUS = 4

def state_to_prompt(state: ANXIETYSTATES) -> str:
    state_mapping = {
        ANXIETYSTATES.OK: "ok",
        ANXIETYSTATES.UNEASY: "uneasy",
        ANXIETYSTATES.CONCERNED: "concerned",
        ANXIETYSTATES.WORRIED: "worried",
        ANXIETYSTATES.ANXIOUS: "anxious",
    }
    return state_mapping[state]

def classify_anxiety_emotion(prompt: str) -> str:
    r = anxiety_emotion_rl(prompt)
    return r.name

def update_anxiety_prompt(user_input: str) -> str:
    global CURRENT_SUB_STATE
    global CURRENT_BASE_STATE
    question_category = classify_anxiety_emotion(user_input)
    print(f"emotional sub route: {question_category}")
    prompt = ""
    if question_category == 'escalate':
        if CURRENT_SUB_STATE != ANXIETYSTATES.ANXIOUS:
            CURRENT_SUB_STATE = ANXIETYSTATES(CURRENT_SUB_STATE.value + 1)
    elif question_category == 'de_escalate':
        if CURRENT_SUB_STATE != ANXIETYSTATES.OK:
            CURRENT_SUB_STATE = ANXIETYSTATES(CURRENT_SUB_STATE.value - 1)
    prompt_key = state_to_prompt(CURRENT_SUB_STATE)
    prompt = emo_prompts[prompt_key]
    if CURRENT_SUB_STATE == ANXIETYSTATES.OK:
        CURRENT_BASE_STATE = 'ok'
    return prompt, question_category

#returns one attitude or belief route
def classify_agent(prompt: str) -> str:
    r = attitude_rl(prompt)
    return r.name

def update_attitude_prompt(user_input: str) -> str:
    question_category = classify_agent(user_input)
    print(f"agent route: {question_category}")
    if question_category is None:
        prompt = "" #default prompt
        route = "No route"
    else:
        prompt = attitude_prompts[question_category]
        route = question_category
    return prompt, route



In [9]:
class ChatBot():  
    def __init__(self, api_key, role): 
        self.client = OpenAI(api_key=api_key)
        self.role = role
        self.messages = [{"role": "user", "content": ""}]
        self.all_conversations = []
    
    def query(self, query: str, emo_prompt: str, ab_prompt: str, emo_route: str, ab_route: str) -> None:
        self.messages.append({"role": "user", "content": query})
        self.messages.insert(0, {"role": "system", "content": self.role + scenario_prompt + emo_prompt + ab_prompt})
        try:
            stream = self.client.chat.completions.create(
                model="gpt-4o-2024-08-06", messages=self.messages,
                stream=True,
            )
            text = []
            for part in stream:
                if part.choices[0].delta.content is not None:
                    response_part = part.choices[0].delta.content
                    print(response_part, end="", flush=True)
                    text.append(response_part)
            full_reply_content = ''.join([m for m in text if m is not None])
            self.messages.append({"role": "assistant", "content": full_reply_content})
            self.messages.pop(0)
            self.all_conversations.append(f"\nRoutes: {emo_route + ab_route} \nCurrent prompt: {emo_prompt + ab_prompt}")
            self.all_conversations.append(self.messages[-2:])
            print('\n')

        except Exception as e:
            print(f"An error occurred: {e}")

    def save_whole_conversation(self, filename):
        with open(f"transcripts/{filename}", "w") as file:
            for conversation in self.all_conversations:
                if isinstance(conversation, str):
                    file.write(conversation + "\n")
                elif isinstance(conversation, list):
                    for message in conversation:
                        if message["role"] == "user":
                            file.write("User: " + message["content"] + "\n")
                        elif message["role"] == "assistant":
                            file.write("Agent: " + message["content"] + "\n")

#initialises chatbot
rpa = ChatBot(api_key=api_key, role = system_prompt)

In [ ]:
def on_pasted_submit(b):
    pasted_content = pasted_input_field.value
    EMO_PROMPT, EMO_ROUTE = update_emo_prompt(pasted_content)
    print(f"current emo prompt: {EMO_PROMPT}")
    AB_PROMPT, AB_ROUTE = update_attitude_prompt(pasted_content)
    print(f"current attitude prompt: {AB_PROMPT}")
    rpa.query(pasted_content, emo_prompt = EMO_PROMPT, ab_prompt = AB_PROMPT, emo_route = EMO_ROUTE, ab_route = AB_ROUTE)
    time.sleep(1)
    pasted_input_field.value = ""

def on_save_click(b):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{timestamp}.txt"
    rpa.save_whole_conversation(filename)

pasted_input_field = widgets.Textarea(
    placeholder='Type here :)',
    layout=widgets.Layout(width='100%', height='50px')
)
pasted_submit_button = widgets.Button(description='Submit')
pasted_submit_button.style.button_color = 'lightpink'

save_button = widgets.Button(description='Save')
save_button.style.button_color = 'lightgrey'

pasted_submit_button.on_click(on_pasted_submit)
save_button.on_click(on_save_click)

button_layout = HBox([pasted_submit_button, save_button], layout=Layout(justify_content='center'))

display(pasted_input_field, button_layout)

Textarea(value='', layout=Layout(height='50px', width='100%'), placeholder='Type here :)')

emotional route: None
current emo prompt: Maintain conversation with the user.
agent route: None
current attitude prompt: 
- Married to wife, Sophia, for seven years  
- Two kids: Ethan, six years old and Ava, four years old

